In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


In [81]:
df = pd.read_csv('/content/fashion-mnist_train.csv')

In [82]:
df.shape

(60000, 785)

In [83]:
torch.manual_seed(42)

In [84]:
#check is gpu is avaible or not
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [85]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
x = df.drop(columns='label', axis=1)
y = df['label']

In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [88]:
#Standardiztion
x_train = x_train/255.0
x_test = x_test/255.0

In [89]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels =  torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [90]:
train_dataset = CustomDataset(x_train.values, y_train.values)
test_dataset = CustomDataset(x_test.values, y_test.values)

In [91]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, pin_memory=True)

In [92]:
len(train_loader)

1500

In [93]:
class MyNN(nn.Module):
  def __init__(self, input_dim, output_dim, num_hidden_layer, neuron_per_layer):

    super().__init__()
    layers = []

    for i in range(num_hidden_layer):
      layers.append(nn.Linear(input_dim, neuron_per_layer))
      layers.append(nn.BatchNorm1d(neuron_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(0.3))
      input_dim = neuron_per_layer
    layers.append(nn.Linear(neuron_per_layer, output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)

In [94]:
def objetive(trail):
  #next hyperparameter
  num_hidden_layer = trail.suggest_int("num_hiden_layer", 1, 5)
  neuron_per_layer = trail.suggest_int("neuron_per_layer", 8, 128, step=8)
  #model pararemter
  input_dim = x_train.shape[1]
  output_dim = 10

  model = MyNN(input_dim, output_dim, num_hidden_layer, neuron_per_layer).to(device) # Move model to device

  #parameter analysise
  learning_rate = 0.1
  epochs = 100

  #optimizer
  critertion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=learning_rate) # Corrected to optim.SGD

  for epoch in range(epochs):
    model.train() # Set model to training mode
    for data_features, data_labels in train_loader:
      data_features = data_features.to(device)
      data_labels = data_labels.to(device) # Corrected variable name

      output = model(data_features)
      loss = critertion(output, data_labels) # Corrected typo batch_labeks

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  model.eval() # Set model to evaluation mode
  correct = 0
  total = 0
  with torch.no_grad():
    for data_features, data_labels in test_loader:
      data_features = data_features.to(device)
      data_labels = data_labels.to(device)
      outputs = model(data_features)
      _, predicted = torch.max(outputs.data, 1)
      total += data_labels.size(0)
      correct += (predicted == data_labels).sum().item()
  accuracy = correct / total
  return accuracy # Return accuracy for hyperparameter optimization

In [95]:
!pip install optuna

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objetive, n_trials=10)

[I 2025-12-13 18:09:32,870] A new study created in memory with name: no-name-7bd3e314-9aee-4691-bebd-bb1a1e4d6f1a
